In [ ]:
import typing as T
import os

os.environ["ROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
from scipy.stats import linregress, kendalltau, spearmanr
from scipy.sparse import csr
import mlflow

from power_ratings.featurizers import FeatureBase

In [ ]:
# ! pip install nb-black
%load_ext nb_black

In [ ]:
# some assumptions gleaned from prior data analysis

# an overtime adds about 7-9 points on average
OVERTIME_SCORE_BONUS = 7.4

SEASONS = list(range(2000, 2023))
# SEASONS = (2022,)

DATA_PREFIX = "M"

# if we want to use predetermined values instead of distributions,
# we can set this flag
USE_PREDETERMINED = True

if DATA_PREFIX == "M":
    pre_scaler = 0.59
    pre_base = 69.5
else:
    pre_scaler = 0.79
    pre_base = 64.0

params = {
    "overtime_score_discount": OVERTIME_SCORE_BONUS,
    "season": f"[{','.join([str(i) for i in SEASONS])}]",
    "data_prefix": DATA_PREFIX,
    "use_predetermined": USE_PREDETERMINED,
    "pre_scaler": pre_scaler,
    "pre_base": pre_base,
}

mlflow.set_experiment("feature-generation-2022")
mlflow.log_params({k: str(v) for k, v in params.items()})
mlflow.sklearn.autolog()

In [ ]:
# read in game data, team names, and previously-calculated Elo scores for comparison

feature_base = FeatureBase(prefix=DATA_PREFIX)
games_df = pd.read_csv(f"./data/{DATA_PREFIX}RegularSeasonCompactResults.csv")
teamnames = pd.read_csv(f"data/{DATA_PREFIX}Teams.csv")
elo_df = feature_base.elo_features.reset_index()
elo_df["elo"] = elo_df["elo_32_day0_True"]

games_df = games_df[games_df.Season.isin(SEASONS)]
print(games_df.shape)

games_df = pd.merge(
    games_df,
    elo_df,
    how="inner",
    left_on=["Season", "WTeamID"],
    right_on=["Season", "TeamID"],
)
games_df["WTeamID"] = games_df[["Season", "WTeamID"]].apply(
    lambda x: "_".join([str(v) for v in x]), axis=1
)
games_df = games_df.rename(
    columns={
        "elo": "T1elo",
        "WTeamID": "T1TeamID",
        "WScore": "T1Score",
        "WLoc": "T1Loc",
    }
).drop(columns=["TeamID"])
games_df["T1Wins"] = 1.0

games_df["T1Home"] = games_df.T1Loc.apply(lambda x: 1 if x == "H" else 0)
t2home = games_df.T1Loc.apply(lambda x: 1 if x == "A" else 0).values

games_df = pd.merge(
    games_df,
    elo_df,
    how="inner",
    left_on=["Season", "LTeamID"],
    right_on=["Season", "TeamID"],
)
games_df["LTeamID"] = games_df[["Season", "LTeamID"]].apply(
    lambda x: "_".join([str(v) for v in x]), axis=1
)
games_df = games_df.rename(
    columns={
        "elo": "T2elo",
        "LTeamID": "T2TeamID",
        "LScore": "T2Score",
    }
).drop(columns=["TeamID", "DayNum"])

renames = {c: "T2" + c[2:] for c in games_df.columns if c[:2] == "T1" and c != "T1Home"}
renames.update({c: "T1" + c[2:] for c in games_df.columns if c[:2] == "T2"})
inv_games_df = games_df.copy().rename(columns=renames)
inv_games_df["T1Wins"] = 0.0
inv_games_df["T1Home"] = t2home
games_df = (
    pd.concat([games_df, inv_games_df])
    .drop(columns=["T2Loc", "T2Score"])
    .reset_index(drop=True)
)

games_df = games_df[games_df.NumOT < 2]
games_df["T1Score"] = games_df[["T1Score", "NumOT"]].apply(
    lambda x: x[0] - OVERTIME_SCORE_BONUS * x[1], axis=1
)

In [ ]:
def train_model(games_df):
    mean_game_score = int(games_df[games_df.NumOT == 0].T1Score.mean())

    # factorize turns our team IDs into sequential ints like [0, 1, 2, ...]
    t1_idx, teams = pd.factorize(games_df["T1TeamID"], sort=True)
    t2_idx, _ = pd.factorize(games_df["T2TeamID"], sort=True)
    game_ids = games_df.index.values

    # shape of this is taken from Rugby Analytics example here:
    # https://oriolabril.github.io/oriol_unraveled/python/arviz/pymc3/xarray/2020/09/22/pymc3-arviz.html
    coords = {"team": teams, "game": game_ids}

    with pm.Model(coords=coords) as model:
        # constant data
        t1 = pm.Data("t1", t1_idx, dims="game", mutable=True)
        t2 = pm.Data("t2", t2_idx, dims="game", mutable=True)

        #     off_sigma = pm.Uniform("off_sigma", lower=5, upper=15)
        #     def_sigma = pm.Uniform("def_sigma", lower=5, upper=15)
        off_sigma, def_sigma = 10, 10

        # team-specific model parameters
        offense = pm.Normal("offense", mu=50, sigma=off_sigma, dims="team")
        defense = pm.Normal("defense", mu=50, sigma=def_sigma, dims="team")
        team_score_sigma = pm.Normal("team_score_sigma", mu=9, sigma=2, dims="team")

        if not USE_PREDETERMINED:
            scaler = pm.Normal("scaler", mu=0.5, sigma=0.3)
            base = pm.Normal("base", mu=mean_game_score - 5, sigma=2)
        else:
            scaler = pre_scaler
            base = pre_base

        # game win parameters
        #         x0 = pm.Normal("t1off_minus_t2def_weight", mu=0.1, sigma=0.05)
        #         x1 = pm.Normal("t2off_minus_t1def_weight", mu=0.1, sigma=0.05)

        t1_score = pm.Deterministic(
            "score", (offense[t1_idx] - defense[t2_idx]) * scaler + base
        )
        #         t1_win_prob = pm.Deterministic(
        #             "win_prob",
        #             pm.invlogit(
        #                 x0 * (offense[t1_idx] - defense[t2_idx])
        #                 - x1 * (offense[t2_idx] - defense[t1_idx])
        #             ),
        #         )

        # likelihood of observed data
        t1_pts = pm.Normal(
            "t1_points",
            mu=t1_score,
            sigma=team_score_sigma[t1_idx],
            observed=games_df.T1Score,
            dims=("game"),
        )
        #         t1_wins = pm.Bernoulli("t1_wins", p=t1_win_prob, observed=games_df.T1Wins)

        trace = pm.sample(
            750,
            tune=750,
            cores=10,
            return_inferencedata=True,
            target_accept=0.9,
        )
    return trace


def get_ratings_df(trace):
    trace_hdi = az.hdi(trace, hdi_prob=0.9)
    #     trace_hdi_tight = az.hdi(trace, hdi_prob=0.5)
    ratings_df = pd.DataFrame(
        list(
            zip(
                trace_hdi["offense"].team.values,
                trace_hdi["offense"].values[:, 0],
                trace_hdi["offense"].values[:, 1],
                trace_hdi["defense"].values[:, 0],
                trace_hdi["defense"].values[:, 1],
                trace_hdi["team_score_sigma"].values[:, 0],
                trace_hdi["team_score_sigma"].values[:, 1],
            )
        ),
        columns=[
            "FullTeamID",
            "OffensiveRatingLB",
            "OffensiveRatingUB",
            "DefensiveRatingLB",
            "DefensiveRatingUB",
            "ScoreVarianceLB",
            "ScoreVarianceUB",
        ],
    )
    ratings_df["OffensiveRating"] = (
        trace_hdi["offense"].values[:, 0] + trace_hdi["offense"].values[:, 1]
    ) / 2
    ratings_df["DefensiveRating"] = (
        trace_hdi["defense"].values[:, 0] + trace_hdi["defense"].values[:, 1]
    ) / 2
    ratings_df["ScoreVariance"] = (
        trace_hdi["team_score_sigma"].values[:, 0]
        + trace_hdi["team_score_sigma"].values[:, 1]
    ) / 2

    ratings_df["Season"] = ratings_df["FullTeamID"].apply(
        lambda x: int(x.split("_")[0])
    )
    ratings_df["TeamID"] = ratings_df["FullTeamID"].apply(
        lambda x: int(x.split("_")[1])
    )

    return ratings_df

In [ ]:
full_ratings_df = pd.DataFrame()
for year in SEASONS:
    if year in games_df.Season.unique():
        # process each year with past years to lower inter-year variance
        trace = train_model(
            games_df[games_df.Season.isin((year - 3, year - 2, year - 1, year))].copy()
        )
        ratings_df = get_ratings_df(trace)
        ratings_df = ratings_df[ratings_df.Season == year]
        full_ratings_df = pd.concat([full_ratings_df, ratings_df])

In [ ]:
# coords = {"team": ["2003_3102"]}
az.plot_trace(
    trace,
    var_names=[
        "offense",
        #         "team_score_sigma"
        #         "base",
        #         "t1off_minus_t2def_weight",
        #         "t2off_minus_t1def_weight",
    ],
    #     coords=coords,
    compact=False,
)

In [ ]:
frdf = get_ratings_df(trace)

In [ ]:
# az.summary(trace, kind="diagnostics")

In [ ]:
full_ratings_df

In [ ]:
full_ratings_df.groupby("Season").mean()

In [ ]:
# full_ratings_df = full_ratings_df.groupby("FullTeamID").median().reset_index()
joined = pd.merge(
    full_ratings_df,
    elo_df,
    how="inner",
    left_on=["Season", "TeamID"],
    right_on=["Season", "TeamID"],
)
joined = pd.merge(
    joined, teamnames, how="left", left_on=["TeamID"], right_on=["TeamID"]
)

joined = joined.rename(
    columns={"elo_32_day30_True": "EloWithScore", "elo_32_day0_False": "EloWinLoss"}
)
joined["CombinedRating"] = joined["OffensiveRating"] + joined["DefensiveRating"]
output_df = joined[
    [
        "Season",
        "TeamName",
        "CombinedRating",
        "OffensiveRating",
        "DefensiveRating",
        "ScoreVariance",
        "EloWithScore",
        "EloWinLoss",
        "OffensiveRatingLB",
        "OffensiveRatingUB",
        "DefensiveRatingLB",
        "DefensiveRatingUB",
        "ScoreVarianceLB",
        "ScoreVarianceUB",
        "TeamID",
    ]
].copy()
float_cols = output_df.select_dtypes(float).columns
for fc in float_cols:
    output_df[fc] = output_df[fc].apply(np.round, args=[1])

In [ ]:
output_df.groupby("Season").describe()

In [ ]:
output_df.sort_values(["Season", "CombinedRating"], ascending=False)

In [ ]:
output_df.to_csv(f"output/{DATA_PREFIX}_data_interim.csv", index=False)
joined.to_csv(f"data/{DATA_PREFIX}features_interim.csv", index=False)

mlflow.log_artifact(f"output/{DATA_PREFIX}_data_interim.csv")
mlflow.log_artifact(f"data/{DATA_PREFIX}features_interim.csv")

In [ ]:
# joined_sub = joined[~joined.adj_o.isna()]
# print(f"offensive KT: {kendalltau(joined_sub.adj_o, joined_sub.OffensiveRating)}")
# print(f"defensive KT: {kendalltau(joined_sub.adj_d, joined_sub.DefensiveRating)}")
# print(f"rank to off KT: {kendalltau(joined_sub['rank'], joined_sub.OffensiveRating)}")
# print(f"rank to def KT: {kendalltau(joined_sub['rank'], joined_sub.DefensiveRating)}")
# print(
#     f"rank to summed KT: {kendalltau(joined_sub['rank'], joined_sub.DefensiveRating+joined_sub.OffensiveRating)}"
# )
# print(
#     f"elo to summed KT: {kendalltau(joined_sub['elo_16_day0_True'], joined_sub.DefensiveRating+joined_sub.OffensiveRating)}"
# )

In [ ]:
####################################
# Caclulate efficiency features
####################################
from power_ratings.tournament_dataset import df_rename, feature_rename

detailed_df = pd.read_csv(f"./data/{DATA_PREFIX}RegularSeasonDetailedResults.csv")
existing_feature_df = pd.read_csv(
    f"output/{DATA_PREFIX}_data_interim.csv",
    #     usecols=["Season", "TeamID", "DefensiveRating", "OffensiveRating"],
).set_index(["Season", "TeamID"])

In [ ]:
df_for_eff = pd.merge(
    detailed_df,
    feature_rename(existing_feature_df, "W"),
    how="inner",
    left_on=["Season", "WTeamID"],
    right_index=True
)
df_for_eff = pd.merge(
    df_for_eff,
    feature_rename(existing_feature_df, "L"),
    how="inner",
    left_on=["Season", "LTeamID"],
    right_index=True
)
pos = df_rename(df_for_eff, "W", "L")
neg = df_rename(df_for_eff, "L", "W")
df_for_eff = pd.concat((pos, neg))


In [ ]:
df_for_eff["ApproxPoss"] = df_for_eff.apply(
    lambda x: (
        x["T1FGA"]
        - x["T1OR"]
        + x["T1TO"]
        + 0.475 * x["T1FTA"]
        + x["T2FGA"]
        - x["T2OR"]
        + x["T2TO"]
        + 0.475 * x["T2FTA"]
    )
    / 2,
    axis=1,
)
df_for_eff["PossAdjForOT"] = df_for_eff.apply(
    lambda x: x["ApproxPoss"] - 7.5 * x["NumOT"],
    axis=1,
)

df_for_eff["T1PtsPerPossEstimate"] = df_for_eff.apply(
    lambda x: x["T1Score"] / x["ApproxPoss"] ,
    axis=1,
)


In [ ]:
df_for_eff.head()

In [ ]:
from sklearn.linear_model import LinearRegression
from scipy.sparse import csr

team_map = {}
for team in (
    df_for_eff[["Season", "T1TeamID"]]
    .apply(lambda x: f"{x[0]}_{x[1]}", axis=1)
    .unique()
):
    team_map[team] = len(team_map)


def create_pace_feature(df: pd.DataFrame, team_id_map: T.Dict[int, int]):
    data, rows, cols, y = [], [], [], []
    n = 0
    for season, t1, t2, poss_adj in df[
        ["Season", "T1TeamID", "T2TeamID", "PossAdjForOT"]
    ].values:
        ind1 = team_id_map[f"{int(season)}_{int(t1)}"]
        ind2 = team_id_map[f"{int(season)}_{int(t2)}"]
        rows.extend([n, n])
        cols.extend([ind1, ind2])
        data.extend([1, 1])
        y.append(poss_adj)
        n += 1
    x = csr.csr_matrix((data, (rows, cols)), shape=(n, len(team_id_map)))
    lr = LinearRegression().fit(x, y)

    inv = {v: k for k, v in team_id_map.items()}
    pace_data = []
    for n, coef in enumerate(lr.coef_):
        season_team_id = inv[n]
        season, team_id = [int(i) for i in season_team_id.split("_")]
        pace_data.append([season, team_id, np.round(lr.intercept_ + coef, 1)])
    return pd.DataFrame(pace_data, columns=["Season", "TeamID", "TempoEstimate"])


def create_est_pts_per_poss_feature(df: pd.DataFrame, team_id_map: T.Dict[int, int]):
    data, rows, cols, y = [], [], [], []
    n_teams = len(team_id_map)
    n = 0
    for season, t1, t2def, pts_per_estimate in df[
        ["Season", "T1TeamID", "T2DefensiveRating", "T1PtsPerPossEstimate"]
    ].values:
        ind1 = team_id_map[f"{int(season)}_{int(t1)}"]
        rows.extend([n, n])
        cols.extend([ind1, n_teams])
        data.extend([1, t2def])
        y.append(pts_per_estimate)
        n += 1
    x = csr.csr_matrix((data, (rows, cols)), shape=(n, n_teams + 1))
    lr = LinearRegression().fit(x, y)
    preds = lr.predict(x)
    r2 = linregress(y, preds).rvalue ** 2
    kt = kendalltau(y, preds)[0]
    print(lr.intercept_, lr.coef_[-1], r2, kt)

    inv = {v: k for k, v in team_id_map.items()}
    pace_data = []
    for n, coef in enumerate(lr.coef_[:-1]):
        season_team_id = inv[n]
        season, team_id = [int(i) for i in season_team_id.split("_")]
        pace_data.append([season, team_id, np.round(lr.intercept_ + coef, 2)])
    return pd.DataFrame(
        pace_data, columns=["Season", "TeamID", "PossessionEfficiencyFactor"]
    )


pace_df = create_pace_feature(df_for_eff, team_map)
pts_per_poss_df = create_est_pts_per_poss_feature(df_for_eff, team_map)

In [ ]:
new_features = pd.merge(
    existing_feature_df,
    pts_per_poss_df,
    how="left",
    right_on=["Season", "TeamID"],
    left_index=True,
    suffixes=('_old', '')
).set_index(["Season", "TeamID"])
new_features = pd.merge(
    new_features,
    pace_df,
    how="left",
    right_on=["Season", "TeamID"],
    left_index=True,
    suffixes=('_old', '')
)


In [ ]:
new_features.columns

In [ ]:
new_features[
    [
        "Season",
        "TeamID",
        "TeamName",
        "CombinedRating",
        "OffensiveRating",
        "DefensiveRating",
        "EloWithScore",
        "EloWinLoss",
        "PossessionEfficiencyFactor",
        "TempoEstimate",
        "ScoreVariance",
        "OffensiveRatingLB",
        "DefensiveRatingLB",
        "OffensiveRatingUB",
        "DefensiveRatingUB",
    ]
].to_csv(f"output/{DATA_PREFIX}_data_complete.csv", index=False)

mlflow.log_artifact(f"output/{DATA_PREFIX}_data_complete.csv")

In [ ]:
corr_check = new_features.dropna()
print(kendalltau(corr_check.OffensiveRating, corr_check.PossessionEfficiencyFactor))
print(spearmanr(corr_check.OffensiveRating, corr_check.PossessionEfficiencyFactor))
print(
    linregress(corr_check.OffensiveRating, corr_check.PossessionEfficiencyFactor).rvalue
    ** 2
)

In [ ]:
print(kendalltau(corr_check.OffensiveRating, corr_check.DefensiveRating))
print(spearmanr(corr_check.OffensiveRating, corr_check.DefensiveRating))
print(linregress(corr_check.OffensiveRating, corr_check.DefensiveRating).rvalue ** 2)

In [ ]:
print(kendalltau(corr_check.DefensiveRating, corr_check.PossessionEfficiencyFactor))
print(spearmanr(corr_check.DefensiveRating, corr_check.PossessionEfficiencyFactor))
print(
    linregress(corr_check.DefensiveRating, corr_check.PossessionEfficiencyFactor).rvalue
    ** 2
)